In [ ]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 987.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.0 MB/s eta 0:00:00


#Define your AWS access key and secret key

In [ ]:
import boto3
import os


#Creating bucket to store Input streaming data

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# Define your AWS region
AWS_REGION = 'ap-south-1'  # Replace with your desired region

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY,region_name=AWS_REGION)

# Create a bucket
bucket_name = 'raw-data-input'
s3.create_bucket(Bucket=bucket_name,
                 CreateBucketConfiguration={
                     'LocationConstraint': AWS_REGION
                 })

{'ResponseMetadata': {'RequestId': '8TRQDNTWV6QK4J07',
  'HostId': '0f/QMGaePbiiU/Si7OAicmf7YXJFfzaflU6VAwcFZbLZi9WNBMk5UMVf2ZWipg/IwcASCvVPkpc=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '0f/QMGaePbiiU/Si7OAicmf7YXJFfzaflU6VAwcFZbLZi9WNBMk5UMVf2ZWipg/IwcASCvVPkpc=',
   'x-amz-request-id': '8TRQDNTWV6QK4J07',
   'date': 'Sun, 11 Aug 2024 19:45:49 GMT',
   'location': 'http://raw-data-input1.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://raw-data-input1.s3.amazonaws.com/'}

#Creating Streaming Data


In [ ]:
!split -l 2 /content/click_stream.txt click_stream_

#Loading raw data to the S3 bucket

In [ ]:
import sys
import time
s3_resource = boto3.resource('s3',region_name='ap-south-1')
print("Uploading a file ....")
bucket_name= "raw-data-input1"
def func(a,b):
  local_file_name = a
  s3_file_name=b

  s3_resource.Bucket(bucket_name).upload_file(Filename=local_file_name, Key=s3_file_name)
  print("File upload completed!")

file_path=['/content/click_stream_aa','/content/click_stream_ab','/content/click_stream_ac','/content/click_stream_ad','/content/click_stream_ae']
file_name=['aa','ab','ac','ad','ae']
for i in range(5):
  time.sleep(5)
  func(file_path[i],file_name[i])

Uploading a file ....
File upload completed!
File upload completed!
File upload completed!
File upload completed!
File upload completed!


#Creating S3 bucket for storing output

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,
                         aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# Define your AWS region
AWS_REGION = 'ap-south-1'  # Replace with your desired region

# Create an S3 client
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY,region_name=AWS_REGION)

# Create a bucket
bucket_name = 'raw-data-output'
s3.create_bucket(Bucket=bucket_name,
                 CreateBucketConfiguration={
                     'LocationConstraint': AWS_REGION
                 })

{'ResponseMetadata': {'RequestId': 'PV3TSSVFXE887XRX',
  'HostId': 'TYC2kmydzPkE+d3CIWq7S8l4d8caojD9yYD/3cfwVttHbwWhl5BTKNqn9SRtr30G133WdjmA8Xfxy3teaQ1gkQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'TYC2kmydzPkE+d3CIWq7S8l4d8caojD9yYD/3cfwVttHbwWhl5BTKNqn9SRtr30G133WdjmA8Xfxy3teaQ1gkQ==',
   'x-amz-request-id': 'PV3TSSVFXE887XRX',
   'date': 'Sun, 11 Aug 2024 19:58:50 GMT',
   'location': 'http://raw-data-output.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://raw-data-output.s3.amazonaws.com/'}

#AWS Lambda code to combining streaming data(AmazonS3FullAccess : add policy to lambda) + Add pandas layer

In [ ]:
import json
import boto3
import io
import pandas as pd

def lambda_handler(event, context):
    s3 = boto3.client("s3")

    input_bucket = "raw-data-input"
    list_input = ["aa", "ab","ac","ad","ae"]
    dataframes = []
    output_bucket = "raw-data-output"
    output_key = "combined_data.csv"

    for key in list_input:
        response = s3.get_object(Bucket=input_bucket, Key=key)
        csv_content = response['Body'].read().decode('utf-8')
        df = pd.read_csv(io.StringIO(csv_content),header=None)
        dataframes.append(df)

    if dataframes:
        combined_df = pd.concat(dataframes)

        #Data uploading part
        csv_buffer = io.StringIO()
        combined_df.to_csv(csv_buffer, index=False)
        csv_bytes = csv_buffer.getvalue().encode('utf-8')
        s3.put_object(Bucket=output_bucket, Key=output_key, Body=csv_bytes)

#Redshift (Load s3 bucket into redshift)

*   Load s3 bucket into table trial and make it public
*   GRANT SELECT ON TABLE "dev"."public"."trial" TO PUBLIC;



In [ ]:
pip install boto3 psycopg2

In [ ]:
import boto3
import psycopg2
import json

# Initialize a session using Amazon Redshift Serverless
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='ap-south-1'
)

# Get the Redshift Data API client
client = session.client('redshift-data')

# Set the database name, workgroup name, and other required variables
database = 'dev'
workgroup_name = 'default-workgroup'  # Use workgroup name for Redshift Serverless
db_user = 'admin'
password = 'avi.bansal3A'
sql_query = 'SELECT * FROM "dev"."public"."trial";'


# Execute the SQL query using WorkgroupName for Redshift Serverless
response = client.execute_statement(
    Database=database,
    WorkgroupName=workgroup_name,  # Use WorkgroupName instead of ClusterIdentifier
    # DbUser=db_user,
    Sql=sql_query
)

# Get the query ID from the response
query_id = response['Id']

while True:
    status = client.describe_statement(Id=query_id)
    if status['Status'] in ['STARTED', 'SUBMITTED']:
        continue
    elif status['Status'] in ['FINISHED', 'FAILED']:
        break

# Fetch the results
if status['Status'] == 'FINISHED':
    result = client.get_statement_result(Id=query_id)
    records = result['Records']
    for record in records:
        print(record)
else:
    print("Query failed:", status['Status'])
    print(status)

[{'longValue': 1}, {'stringValue': 'John'}, {'stringValue': 'Doe'}, {'stringValue': 'Software Engineer'}, {'stringValue': 'IT'}, {'longValue': 90000}]
[{'longValue': 2}, {'stringValue': 'Jane'}, {'stringValue': 'Smith'}, {'stringValue': 'Data Analyst'}, {'stringValue': 'Marketing'}, {'longValue': 85000}]
[{'longValue': 3}, {'stringValue': 'Michael'}, {'stringValue': 'Brown'}, {'stringValue': 'Project Manager'}, {'stringValue': 'Operations'}, {'longValue': 95000}]
[{'longValue': 4}, {'stringValue': 'Emily'}, {'stringValue': 'Davis'}, {'stringValue': 'UX Designer'}, {'stringValue': 'Design'}, {'longValue': 80000}]
[{'longValue': 5}, {'stringValue': 'Daniel'}, {'stringValue': 'Wilson'}, {'stringValue': 'Product Manager'}, {'stringValue': 'Product'}, {'longValue': 100000}]
[{'longValue': 6}, {'stringValue': 'Sarah'}, {'stringValue': 'Johnson'}, {'stringValue': 'HR Manager'}, {'stringValue': 'Human Resources'}, {'longValue': 75000}]
[{'longValue': 7}, {'stringValue': 'James'}, {'stringValue